In [1]:
import preprocess as pre
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, TimeDistributed
from tensorflow.keras.layers import Concatenate, Flatten
from tensorflow.keras.layers import GRU, Conv2D, MaxPooling2D, Embedding
from tensorflow.keras.layers import Input, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras

In [2]:
from data_gen import *
from models import *

In [3]:
char2int, feat2val, max_r, max_w = pre.process()
data = pre.convert(char2int, feat2val, max_r, max_w)

In [4]:
int2char = {val: key for val, key in enumerate(char2int)}

In [5]:
batch_size = 128

In [6]:
gen = pre.gen(data, batch_size)

In [7]:
max_root = max_r + 2
max_word = max_w + 2
n_feature = data[1].shape[1]
hidden_size = 256
feat_embed_size = 32
char_embed_size = 32
n_batches = len(data[0]) // batch_size
print("Total Data: {0} Total Batches {1}".format(len(data[0]), n_batches))

Total Data: 120121 Total Batches 938


In [8]:
def embed_model(vocab_size, input_length, decoder_input, dec_output_length, 
                    n_feature, hidden_size, feat_units = 15, embed_size=64):
    root_word_input = Input(shape=(input_length, ), name="root_word_input")
    feature_input = Input(shape=(n_feature,), name="word_feature_input")

    feat_out = Dense(feat_units, activation="relu", name="feature_embedding")(feature_input)
    embedding = Embedding(vocab_size, embed_size, input_length=input_length, name="char_embedding")
    x = embedding(root_word_input)
    
    x = Reshape([45, 32, 1])(x)
#     x = keras.backend.expand_dims(x, -1)
    x = Conv2D(32, (5, 5), padding='same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)

    x = Flatten()(x)
    x = Dense(hidden_size - feat_units, activation='relu')(x)
    state_h = Concatenate()([x, feat_out])
    
    decoder_inputs = Input(shape=(None, ), name="target_word_input")
    emb_dec = embedding(decoder_inputs)
    decoder_gru = GRU(hidden_size, return_sequences=True, return_state=True, name="decoder_gru")
    decoder_outputs, _= decoder_gru(emb_dec, initial_state=state_h)
    
    decoder_dense = Dense(dec_output_length, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([root_word_input, feature_input, decoder_inputs], decoder_outputs)
    encoder_model = Model([root_word_input, feature_input], state_h)
    
    decoder_state_input_h = Input(shape=(hidden_size,))
    decoder_outputs, state_h= decoder_gru(emb_dec, initial_state=decoder_state_input_h)

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs, decoder_state_input_h], [
                          decoder_outputs, state_h])

    return model, encoder_model, decoder_model

In [9]:
model, encoder, decoder = embed_model(vocab_size=len(char2int), input_length=max_root, 
                                      decoder_input=max_word, dec_output_length=len(char2int),
                                      n_feature=n_feature, hidden_size=hidden_size, 
                                      feat_units=feat_embed_size, embed_size=char_embed_size)
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [10]:
history = model.fit_generator(gen, steps_per_epoch=n_batches, epochs = 10)

W0725 00:29:26.159852  1884 deprecation.py:323] From C:\Users\amany\.conda\envs\tf2\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
938/938 [==============================] - 141s 150ms/step - loss: 0.7631
Epoch 2/10
938/938 [==============================] - 137s 146ms/step - loss: 0.3138
Epoch 3/10
938/938 [==============================] - 136s 145ms/step - loss: 0.1971
Epoch 4/10
938/938 [==============================] - 136s 145ms/step - loss: 0.1446
Epoch 5/10
938/938 [==============================] - 137s 146ms/step - loss: 0.1134
Epoch 6/10
938/938 [==============================] - 138s 147ms/step - loss: 0.0932
Epoch 7/10
938/938 [==============================] - 138s 147ms/step - loss: 0.0793
Epoch 8/10
938/938 [==============================] - 138s 147ms/step - loss: 0.0685
Epoch 9/10
938/938 [==============================] - 137s 146ms/step - loss: 0.0607
Epoch 10/10
938/938 [==============================] - 136s 145ms/step - loss: 0.0544


In [11]:
test_data = pre.convert(char2int, feat2val, max_root - 2, max_word - 2, train_set=False)
test_gen = pre.gen(test_data, batch_size)


In [12]:
def predict(infenc, infdec, inputs, n_steps):
    # encode
    state = infenc.predict(inputs)
#     start of sequence input
    target_seq = np.array([char2int['<']]*len(inputs[0])).reshape((len(inputs[0]), 1))
#     start[0] = 1
#     target_seq = np.array(start).reshape(1, 1, cardinality)
    # collect predictions
    outputs = list()
    for t in range(n_steps):
        # predict next char
        yhat, h = infdec.predict([target_seq, state])
        # store prediction
#         output.append(yhat[0,0,:])
        # update state
        state = h
#         print(yhat.shape)
        # update target sequence
        target_seq = yhat.argmax(axis=2)
#         print(target_seq.shape)
        outputs.append(target_seq)
    return np.stack(outputs)

In [13]:
# shows sample examples and calculates accuracy
test_batches = len(test_data[0]) // batch_size
total, correct = 0, 0
in_word = 0
sims = []
for b in range(1):
    # get data from test data generator
    [root, feat, dec_in], y = next(test_gen)
    pred = predict(encoder, decoder, [root, feat], max_word)
    for k in range(batch_size):
        indexes = pred[:, k].flatten()
        r = ''.join(pre.index_to_word(root[k], int2char)).strip()[1:-1]
        w = ''.join(pre.index_to_word(dec_in[k], int2char)).strip()[1:-1]
        t = ''.join(pre.index_to_word(indexes, int2char)).strip()[:-1]
        if w == t:
            correct += 1
        

    total += batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Exact Accuracy: 61.72%
